# Auto ML using TPOT

This time, I am using Auto ML.
Let's see which model will it choose under 10 minutes timeframe.

In [1]:
# import required libraries
import sys, os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, make_scorer
from tpot import TPOTRegressor

local_module_path = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()),'preprocess'))
sys.path.append(local_module_path)
from pscript import preprocess

In [2]:
# load the dataset
train = pd.read_csv("../../../dataset/train.csv")
test = pd.read_csv("../../../dataset/test.csv")

In [3]:
# preprocess the datasets
ptrain = preprocess(train)
ptest = preprocess(test)

In [4]:
# ready dataset for training and testing
rtrain = ptrain.drop(['id','accident_risk'], axis=1)
target = ptrain['accident_risk']

rtest = ptest.drop('id', axis=1)

In [5]:
# optimization methods
def rmse_func(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse_func, greater_is_better=False)

In [6]:
# initialize the model
model = TPOTRegressor(verbose=5, max_eval_time_mins=20, max_time_mins=600, n_jobs=1, scorers=rmse_scorer)

In [7]:
# train the model
model.fit(rtrain, target)

Generation: : 0it [00:00, ?it/s]

WARNING AN INDIVIDUAL TIMED OUT (Fallback): 
 {'individual': <tpot.search_spaces.pipelines.sequential.SequentialPipelineIndividual object at 0x00000201C4FD7770>, 'time': 1759518406.2694054} 

WARNING AN INDIVIDUAL TIMED OUT (Fallback): 
 {'individual': <tpot.search_spaces.pipelines.sequential.SequentialPipelineIndividual object at 0x00000201C4FD7070>, 'time': 1759520807.4103932} 

WARNING AN INDIVIDUAL TIMED OUT (Fallback): 
 {'individual': <tpot.search_spaces.pipelines.sequential.SequentialPipelineIndividual object at 0x00000201C4FD6C80>, 'time': 1759523208.3476698} 

WARNING AN INDIVIDUAL TIMED OUT (Fallback): 
 {'individual': <tpot.search_spaces.pipelines.sequential.SequentialPipelineIndividual object at 0x00000201C4FD6820>, 'time': 1759525608.8653035} 

WARNING AN INDIVIDUAL TIMED OUT (Fallback): 
 {'individual': <tpot.search_spaces.pipelines.sequential.SequentialPipelineIndividual object at 0x00000201C4FD6200>, 'time': 1759528009.4892542} 

WARNING AN INDIVIDUAL TIMED OUT (Fallbac

Generation: : 1it [10:06:35, 36395.77s/it]

Generation:  1
Best rmse_func score: -0.14573379569569048



2025-10-04 10:37:10,245 - distributed.scheduler - ERROR - Removing worker 'tcp://127.0.0.1:61075' caused the cluster to lose scattered data, which can't be recovered: {'DataFrame-555f5e0123b2af26ee7fb3b65cc82ff5', 'Series-8c5704e5a54e54c0ddec290b861a7800'} (stimulus_id='handle-worker-cleanup-1759554430.242352')
2025-10-04 10:37:14,030 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2025-10-04 10:37:14,353 - distributed.diskutils - ERROR - Failed to remove 'C:\\Users\\Shivam\\AppData\\Local\\Temp\\dask-scratch-space\\scheduler-88u1a7u6' (failed in <built-in function lstat>): [WinError 2] The system cannot find the file specified: 'C:\\Users\\Shivam\\AppData\\Local\\Temp\\dask-scratch-space\\scheduler-88u1a7u6'


TPOTRegressor(max_eval_time_mins=20, max_time_mins=600,
              scorers=make_scorer(rmse_func, greater_is_better=False, response_method='predict'),
              search_space=<tpot.search_spaces.pipelines.sequential.SequentialPipeline object at 0x00000201C3F1C830>,
              verbose=5)

In [8]:
print(model.fitted_pipeline_)

Pipeline(steps=[('normalizer', Normalizer(norm=np.str_('l1'))),
                ('variancethreshold',
                 VarianceThreshold(threshold=0.0013003990841)),
                ('featureunion-1',
                 FeatureUnion(transformer_list=[('featureunion',
                                                 FeatureUnion(transformer_list=[('rbfsampler',
                                                                                 RBFSampler(gamma=0.359066603675,
                                                                                            n_components=4))])),
                                                ('passthrough',
                                                 Passthrough())])),
                ('featureunion-2',
                 FeatureUnion(transformer_list=[('skiptransformer',
                                                 SkipTransformer()),
                                                ('passthrough',
                                            

In [9]:
# predict the values
preds = model.predict(rtest)

c:\Users\Shivam\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [10]:
prediction = []
for x in preds:
    prediction.append(round(x,3))

In [11]:
result = pd.DataFrame({
    'id': ptest['id'],
    'accident_risk': prediction
})

In [12]:
result.to_csv("../submissions/submission05.csv", index=False)